<a href="https://colab.research.google.com/github/yuvanshanka4/dataaces_works/blob/main/day2_yuvan_summ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install langchain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [1]:
!pip install transformers[sentencepiece]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00


In [19]:
file = open("/content/sample.txt", "r")
FileContent = file.read().strip()
len(FileContent)

15871

In [20]:

# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)



In [21]:
tokenizer.model_max_length
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence
# number of special tokens
tokenizer.num_special_tokens_to_add()


2

In [22]:

# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


47

In [23]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

4

In [24]:
[len(tokenizer.tokenize(c)) for c in chunks]

[len(tokenizer(c).input_ids) for c in chunks]

sum([len(tokenizer(c).input_ids) for c in chunks])

len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (3312 > 1024). Running this sequence through the model will result in indexing errors


3312

In [30]:
import warnings

warnings.filterwarnings("ignore")
# Your code generating the warnings goes here
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Lily was an orphan who had been raised in the town's orphanage. She would often sneak out in the middle of the night to explore the nearby woods and imagine herself in far-off lands. The once peaceful realm was facing dark times, as a malevolent sorceress named Morgana had been sowing discord and chaos. The key to restoring harmony in Serendell lay in a selfless act of love and sacrifice.
 The "Heartstone" is a powerful gem imbued with the purest form of magic, capable of healing any wounds, mending any broken bonds, and restoring balance to all realms. With her newfound friends, Lily set out on a new quest to find the Heartstone. The journey took them to distant lands beyond Serendell, where they encountered new challenges.
 The once vibrant and magical Serendell was now a place of haunting silence and eerie desolation. A curse had been awakened by the absence of the Heartstone, the source of the realm's balance and magic. With the help of a few remaining allies from her previous adv